# Fluxnet Hainich
> Load Hainich dataset from fluxnet

Due to license reason the data is not included in the repo, but you can download it from fluxnet

In [ ]:
#| hide
#| default_exp data

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#| export
from pathlib import Path
from pyprojroot import here
import pandas as pd
import numpy as np

In [ ]:
#| exports
_def_meteo_vars = {
    "TA_F": "TA",
    "SW_IN_F": "SW_IN",
    # "LW_IN_F": "LW_IN",
    "VPD_F": "VPD",
    #"PA": "PA"
}


units = {
    'TA': '°C',
    'SW_IN': 'W m-2',
    # 'LW_IN': 'W m-2',
    'VPD': 'hPa'
}

hai_path_raw = here("data") / "FLX_DE-Hai_FLUXNET2015_FULLSET_HH_2000-2012_1-4.csv"
hai_path = here("data") / "FLX_DE-Hai_FLUXNET2015_FULLSET_HH_2000-2012_1-4_float32.parquet"
hai_path64 = here("data") / "FLX_DE-Hai_FLUXNET2015_FULLSET_HH_2000-2012_1-4_float64.parquet"

In [ ]:
#| exporti

def get_dtype(col_name: str, num_dtype=np.float32):
    "Get correct dtype based on column name"
    if col_name in ["TIMESTAMP_END", "TIMESTAMP_START"]:
        return 'str'
    elif col_name.endswith("QC"):
        return None # pd.CategoricalDtype
    else:
        return num_dtype

def col_types(cols, num_dtype=np.float32):
    return {col: get_dtype(col, num_dtype) for col in cols}

def read_col_names(path):
    "read only column names from csv"
    return pd.read_csv(path, nrows=0).columns

In [ ]:
col_types(read_col_names(hai_path_raw)[:10]) # only for 10 cols for testing

{'TIMESTAMP_START': 'str',
 'TIMESTAMP_END': 'str',
 'TA_F_MDS': numpy.float32,
 'TA_F_MDS_QC': None,
 'TA_ERA': numpy.float32,
 'TA_F': numpy.float32,
 'TA_F_QC': None,
 'SW_IN_POT': numpy.float32,
 'SW_IN_F_MDS': numpy.float32,
 'SW_IN_F_MDS_QC': None}

In [ ]:
#| export
def read_fluxnet_csv(path,
                     nrows:int,
                     meteo_vars: dict[str, str] = _def_meteo_vars,
                     num_dtype = np.float32 # type for numerical columns
                    ):
    "Read fluxnet csv in Pandas with correct parsing of csv"
    return (pd.read_csv(path, na_values=["-9999", "-9999.99"],
                        parse_dates=[0, 1],
                        nrows=nrows,
                        dtype=col_types(read_col_names(path), num_dtype)
                       )
           .rename(columns=meteo_vars)
           .rename(columns={'TIMESTAMP_END': "time"})
           .set_index("time")
           .loc[:, meteo_vars.values()])

hainich default df

In [ ]:
hai_path_raw

PosixPath('/home/simone/Documents/uni/Thesis/GPFA_imputation/data/FLX_DE-Hai_FLUXNET2015_FULLSET_HH_2000-2012_1-4.csv')

In [ ]:
%time hai = read_fluxnet_csv(hai_path_raw, None)

CPU times: user 20.8 s, sys: 228 ms, total: 21.1 s
Wall time: 21.2 s


In [ ]:
hai.to_parquet(hai_path)

In [ ]:
%time hai64 = read_fluxnet_csv(hai_path_raw, None, num_dtype=np.float64)

CPU times: user 20.5 s, sys: 201 ms, total: 20.7 s
Wall time: 20.8 s


In [ ]:
hai64.to_parquet(hai_path64)

In [ ]:
%time pd.read_parquet(hai_path64)

CPU times: user 12.8 ms, sys: 5.97 ms, total: 18.8 ms
Wall time: 12.5 ms


,TA,SW_IN,VPD
time,,,
2000-01-01 00:30:00,-0.60,0.0,0.222
2000-01-01 01:00:00,-0.65,0.0,0.122
2000-01-01 01:30:00,-0.58,0.0,0.090
2000-01-01 02:00:00,-0.51,0.0,0.110
2000-01-01 02:30:00,-0.49,0.0,0.102
...,...,...,...
2012-12-31 22:00:00,4.75,0.0,2.249
2012-12-31 22:30:00,4.48,0.0,2.154
2012-12-31 23:00:00,4.32,0.0,2.108


In [ ]:
#| export
try:
    hai = pd.read_parquet(hai_path)
except FileNotFoundError: # for CI
    hai = pd.DataFrame()

In [ ]:
hai.dtypes

TA       float32
SW_IN    float32
VPD      float32
dtype: object

In [ ]:
hai

,TA,SW_IN,VPD
time,,,
2000-01-01 00:30:00,-0.60,0.0,0.222
2000-01-01 01:00:00,-0.65,0.0,0.122
2000-01-01 01:30:00,-0.58,0.0,0.090
2000-01-01 02:00:00,-0.51,0.0,0.110
2000-01-01 02:30:00,-0.49,0.0,0.102
...,...,...,...
2012-12-31 22:00:00,4.75,0.0,2.249
2012-12-31 22:30:00,4.48,0.0,2.154
2012-12-31 23:00:00,4.32,0.0,2.108


## ERA

In [ ]:
#| export
hai_era_path_raw = here("data") / "FLX_DE-Hai_FLUXNET2015_ERAI_HH_1989-2014_1-4.csv"
hai_era_path = here("data")/"FLX_DE-Hai_FLUXNET2015_ERAI_HH_1989-2014_1-4_float32.parquet"
hai_era_path64 = here("data")/"FLX_DE-Hai_FLUXNET2015_ERAI_HH_1989-2014_1-4_float64.parquet"

In [ ]:
_def_meteo_vars

{'TA_F': 'TA', 'SW_IN_F': 'SW_IN', 'VPD_F': 'VPD'}

In [ ]:
#| export
era_vars = {
    'TA_ERA': 'TA_ERA',
    'SW_IN_ERA': 'SW_IN_ERA',
    'VPD_ERA': 'VPD_ERA'
}

In [ ]:
%time hai_era = read_fluxnet_csv(hai_era_path_raw, None, meteo_vars = era_vars)

CPU times: user 35 s, sys: 33.6 ms, total: 35 s
Wall time: 35.2 s


In [ ]:
hai_era.to_parquet(hai_era_path)

In [ ]:
%time hai_era = pd.read_parquet(hai_era_path)

CPU times: user 10.6 ms, sys: 11 ms, total: 21.6 ms
Wall time: 13.1 ms


In [ ]:
hai_era64 = read_fluxnet_csv(hai_era_path_raw, None, meteo_vars = era_vars, num_dtype=np.float64)

In [ ]:
hai_era64.to_parquet(hai_era_path64)

## Export 

In [ ]:
#| hide
from nbdev import nbdev_export
nbdev_export()